In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.parallel
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import argparse
import scipy.io
import os

class MLP(nn.Module):
  def __init__(self, input_size, output_size, dropout=0.002, num_layers = 3, num_neurons = 4096):
        super(MLP, self).__init__()
        self.num_layers = num_layers
        self.layers = nn.ModuleList()
        self.bn_layers = nn.ModuleList()
        self.activation_fn = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        for i in range(num_layers):
          if i == 0:
            self.layers.append(nn.Linear(input_size, num_neurons))
            self.bn_layers.append(nn.BatchNorm1d(num_neurons))
          else:
            self.layers.append(nn.Linear(num_neurons // (2**(i-1)), num_neurons // (2**i)))
            self.bn_layers.append(nn.BatchNorm1d(num_neurons // (2**i)))

        # Try Relu also here after going through hyperparamter tuning
        self.output_layer = nn.Linear(num_neurons // (2**(num_layers-1)), output_size)
        self.dropout = nn.Dropout(dropout)

  def forward(self, x):
      for i in range(self.num_layers):
        x = self.layers[i](x)
        x = self.bn_layers[i](x)
        x = self.dropout(x)
        x = self.activation_fn(x)

      x = self.output_layer(x)
      x = self.sigmoid(x)
      return x

mat_data = scipy.io.loadmat('Porsche/5000_scaled_3LVL_properties_data.mat')

#Get scaled input and scaled output data
#Min max Scaling
input = mat_data.get('input')
output = mat_data.get('output')
input_scaled = mat_data.get('input_scaled')
output_scaled = mat_data.get('output_scaled')

input_size = input_scaled .shape[1]
output_size = output_scaled .shape[1]
# Load the trained model from the .pt file
model = MLP(input_size, output_size)  # Replace input_size and output_size with appropriate values
path = 'Porsche/best_model_r_square_0.99823_loss_MSE_optimizer_ADAM_lr_0.0004_dropout_0002_3LVL_ANPC.pt'
model.load_state_dict(torch.load(path)) #, map_location=torch.device('cpu')))
model.eval()

# Prepare the input data for prediction
input_data = torch.Tensor(input_scaled)
output_data = torch.Tensor(output_scaled)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Split the data into training, validation, and testing sets
#test_size = 0.2 ist 80% Training set
train_data, test_data, train_target, test_target = train_test_split(
    input_data, output_data, test_size=0.5, random_state=42
)

#test_size hier auf 0.5 heißt die übrigen 20% nochmal in 10% für val und 10% für test
val_data, test_data, val_target, test_target = train_test_split(
    test_data, test_target, test_size=0.5, random_state=42
)
#data = pd.DataFrame(train_data.numpy())
#print(len(data.iloc[:,7]))
#unique_values = data.iloc[:,7].unique()
#print(len(unique_values))
# Create data loaders
train_dataset = TensorDataset(train_data, train_target)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(val_data, val_target)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataset = TensorDataset(test_data, test_target)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

cuda


In [4]:
#RMSE Error metric
#RMSE Error shows the biggest difference of predicted value to original value
model.to(device)
criterion = nn.MSELoss()
model.eval()
test_loss = 0.0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

test_loss /= len(test_loader)
rmse = test_loss ** 0.5  # Calculate the square root of the MSE
print(f"Test RMSE: {rmse:.4f}")
#print(f"Test MSE: {test_loss:.4f}")

Test RMSE: 0.0025


In [5]:
#MAE (Mean Absolute Error) Error metric
#Shows you the average difference between the predicted values and the original values
model.eval()
test_mae = 0.0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        mae = torch.mean(torch.abs(outputs - labels))
        test_mae += mae.item()

test_mae /= len(test_loader)
print(f"Test MAE: {test_mae:.4f}")

Test MAE: 0.0010


In [6]:
#R2-Score metric
#If R-square equals 1, it means that the model perfectly fits
#the data and there is no difference between the predicted value and actual value!
model.eval()
predictions = []
targets = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        predictions.append(outputs.cpu().numpy())
        targets.append(labels.cpu().numpy())

predictions = np.concatenate(predictions, axis=0)
targets = np.concatenate(targets, axis=0)

#Adjusted R2-Score
def adjusted_r2_score(r2, n_samples, n_features):
    adj_r2 = 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_features - 1)
    return adj_r2

# Get the number of samples and features
n_samples = targets.shape[0]
n_features = inputs.shape[1]  # Replace 'inputs' with the appropriate variable containing your input data

# Calculate R2 score
r2 = r2_score(targets, predictions)

# Calculate Adjusted R2 score
adj_r2 = adjusted_r2_score(r2, n_samples, n_features)

# Print both R2 and Adjusted R2 scores
print(f"R^2 Score: {r2:.4f}")
print(f"Adjusted R^2 Score: {adj_r2:.4f}")

R^2 Score: 0.9982
Adjusted R^2 Score: 0.9982
